## База данных: РОСАТОМ

# Модель БД и заполнение

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import (MetaData, Table, Column, Integer, Float, Numeric, String, DateTime, 
                        Boolean, ForeignKey, create_engine, PrimaryKeyConstraint, 
                        UniqueConstraint, CheckConstraint, ForeignKeyConstraint,
                        Index, insert, BigInteger, text)
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker, scoped_session
import requests
import io
from datetime import datetime

Модель данных
![](https://avatars.mds.yandex.net/get-images-cbir/4581702/pts04KF6_VSoP3M_8TijeQ6722/ocr)


In [3]:
# Создадим базу данных согласно модели
engine = create_engine('sqlite:///ROSATOM.db')

metadata = MetaData()

fact_nums=Table('fact_nums', metadata,
             Column('org_id', Integer(), primary_key=True),
             Column('org_division_id', Integer()),
             Column('indicator_id', Integer(), primary_key=True),
             Column('indicator_type_id', Integer()),
             Column('statement_id', Integer()),
             Column('num17', Float(precision=32, decimal_return_scale=None)),
             Column('num18', Float(precision=32, decimal_return_scale=None)),
             Column('num19', Float(precision=32, decimal_return_scale=None)), 
             Column('num20', Float(precision=32, decimal_return_scale=None)),
             Column('num21', Float(precision=32, decimal_return_scale=None)),   
             extend_existing=True)

dict_indicator=Table('dict_indicator', metadata,
            Column('indicator_id', Integer(), ForeignKey('fact_nums.indicator_id'), primary_key=True),
            Column('indicator_name', String()),
             extend_existing=True 
            )

dict_indicator_type=Table('dict_indicator_type', metadata,
            Column('indicator_type_id', Integer(), ForeignKey('fact_nums.indicator_type_id'), primary_key=True),
            Column('indicator_type_name', String()),
             extend_existing=True 
            )

dict_statement=Table('dict_statement', metadata,
            Column('statement_id', Integer(), ForeignKey('fact_nums.statement_id'), primary_key=True),
            Column('statement_name', String()),
             extend_existing=True 
            )

dict_org=Table('dict_org', metadata,
           Column('org_id', Integer(), ForeignKey('fact_nums.org_id'), primary_key=True),
           Column('org_name', String()),
           extend_existing=True
           )

dict_org_division=Table('dict_org_division', metadata,
           Column('org_division_id', Integer(), ForeignKey('fact_nums.org_division_id'), primary_key=True),
           Column('org_division_name', String()),
           extend_existing=True
           )

metadata.create_all(engine)

In [4]:
# Функция по вставке значений в таблицу БД
def table_insert(table_name, df_name):
  bookins = []
  for i in range(len(df_name)):
    bookins.append(df_name.loc[i].to_dict())
  bookinsert = table_name.insert()
  connection = engine.connect()
  result = connection.execute(bookinsert, bookins)
  print(result.inserted_primary_key_rows) #Вывод значений в ключевых столбцах

In [5]:
# Загрузка данных
url = "https://raw.githubusercontent.com/sia-hub/rosatom/main/Data.xlsx"
df_names = pd.ExcelFile(url).sheet_names
for i, sheet in enumerate(df_names):
  globals()['df_'+sheet] = pd.read_excel(url, sheet_name = i)

# Заполним базу данных
for sheet in df_names:
  print(sheet)
  table_insert(globals()[sheet], globals()['df_'+sheet])

fact_nums
[(1.0, 129.0), (1.0, 1.0), (1.0, 2.0), (1.0, 3.0), (1.0, 4.0), (1.0, 5.0), (1.0, 6.0), (1.0, 10.0), (1.0, 11.0), (1.0, 12.0), (1.0, 13.0), (1.0, 14.0), (1.0, 15.0), (1.0, 16.0), (1.0, 17.0), (1.0, 18.0), (1.0, 19.0), (1.0, 21.0), (1.0, 22.0), (1.0, 25.0), (1.0, 24.0), (1.0, 26.0), (1.0, 27.0), (1.0, 29.0), (1.0, 30.0), (1.0, 31.0), (1.0, 32.0), (1.0, 33.0), (1.0, 34.0), (1.0, 35.0), (1.0, 36.0), (1.0, 37.0), (1.0, 38.0), (1.0, 39.0), (1.0, 40.0), (1.0, 41.0), (1.0, 42.0), (1.0, 44.0), (1.0, 43.0), (1.0, 45.0), (1.0, 46.0), (1.0, 47.0), (1.0, 48.0), (1.0, 49.0), (1.0, 50.0), (1.0, 51.0), (1.0, 52.0), (1.0, 53.0), (1.0, 54.0), (1.0, 56.0), (1.0, 58.0), (1.0, 60.0), (1.0, 61.0), (1.0, 62.0), (1.0, 63.0), (1.0, 64.0), (1.0, 65.0), (1.0, 66.0), (1.0, 68.0), (1.0, 69.0), (1.0, 70.0), (1.0, 71.0), (1.0, 72.0), (1.0, 73.0), (1.0, 75.0), (1.0, 76.0), (1.0, 77.0), (1.0, 78.0), (1.0, 80.0), (1.0, 82.0), (1.0, 83.0), (1.0, 84.0), (1.0, 85.0), (1.0, 86.0), (1.0, 89.0), (1.0, 90.0), (1.0, 

# Обращение к БД

In [ ]:
# https://stackoverflow.com/questions/17972020/how-to-execute-raw-sql-in-flask-sqlalchemy-app 
# https://hackersandslackers.com/connecting-pandas-to-a-sql-database-with-sqlalchemy/ - топ

Модель данных
![](https://avatars.mds.yandex.net/get-images-cbir/4581702/pts04KF6_VSoP3M_8TijeQ6722/ocr)

In [6]:
# Главная функция для SQL
def sql(zapros):
  return pd.read_sql(zapros, con=engine)

In [8]:
sql('''
SELECT 
fn.org_id,
fn.org_division_id,
dod.org_division_name,
fn.indicator_id,
di.indicator_name,
fn.indicator_type_id,
dit.indicator_type_name,
fn.statement_id,
ds.statement_name,
fn.num17,
fn.num18,
fn.num19,
fn.num20,
fn.num21

FROM fact_nums fn
JOIN dict_org do ON fn.org_id = do.org_id 
JOIN dict_org_division dod ON fn.org_division_id = dod.org_division_id
JOIN dict_indicator di ON fn.indicator_id = di.indicator_id 
JOIN dict_indicator_type dit ON fn.indicator_type_id = dit.indicator_type_id
JOIN dict_statement ds ON fn.statement_id = ds.statement_id




''')

,org_id,org_division_id,org_division_name,indicator_id,indicator_name,indicator_type_id,indicator_type_name,statement_id,statement_name,num17,num18,num19,num20,num21
0,1,22,Электроэнергетический,129,Cooтношение дебиторской задолженности к актива...,1,%,0,Нет,5.640000e-02,8.360000e-02,1.115000e-01,3.740000e-02,1.636000e-01
1,1,22,Электроэнергетический,1,Активы всего,0,RUB,1,Баланс,1.267406e+09,1.337535e+09,1.495419e+09,1.524282e+09,1.964642e+09
2,1,22,Электроэнергетический,2,Валовая прибыль (убыток),0,RUB,2,Отчёт о финансовых результатах,5.039000e+06,3.279700e+07,5.956000e+06,6.881000e+06,1.106100e+07
3,1,22,Электроэнергетический,3,Валовая рентабельность затрат,1,%,0,Нет,2.950000e-02,1.725000e-01,2.890000e-02,5.600000e-02,2.750000e-02
4,1,22,Электроэнергетический,4,Валовая рентабельность коммерческих и управлен...,1,%,0,Нет,1.607000e-01,1.097000e+00,2.123000e-01,1.888000e-01,-3.579000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27214,307,12,Наука и инновации,122,Соотношение совокупного долга к EBIT,1,%,0,Нет,0.000000e+00,0.000000e+00,0.000000e+00,-4.913470e+04,3.554551e+04
27215,307,12,Наука и инновации,121,Соотношение совокупного долга к капиталу,1,%,0,Нет,0.000000e+00,0.000000e+00,0.000000e+00,2.325300e+00,1.455430e+01
27216,307,12,Наука и инновации,123,Соотношение чистого долга к капиталу,1,%,0,Нет,0.000000e+00,0.000000e+00,-1.065400e+00,2.264100e+00,1.455250e+01
27217,307,12,Наука и инновации,124,Текущий налог на прибыль,0,RUB,2,Отчёт о финансовых результатах,0.000000e+00,0.000000e+00,-1.440000e+05,-5.009000e+06,-1.603000e+06


In [9]:
sql('''
 --Вывод показателя и его измерителя
  
SELECT DISTINCT
fn.indicator_id,
di.indicator_name,
dit.indicator_type_name

FROM fact_nums fn
JOIN dict_indicator di ON fn.indicator_id = di.indicator_id 
JOIN dict_indicator_type dit ON fn.indicator_type_id = dit.indicator_type_id

''')

,indicator_id,indicator_name,indicator_type_name
0,1,Активы всего,RUB
1,2,Валовая прибыль (убыток),RUB
2,3,Валовая рентабельность затрат,%
3,4,Валовая рентабельность коммерческих и управлен...,%
4,5,Валовая рентабельность,%
...,...,...,...
124,59,Переоценка внеоборотных активов,RUB
125,79,"Поступления от выпуска облигаций, векселей и д...",RUB
126,9,Выкуп акций (долей) у собственников,RUB
127,91,"Приобретение основных средств, инвентаря и ино...",RUB


In [10]:
sql('''
 --Расчёт показателей по годам по индексу

WITH t1 AS (SELECT org_id, indicator_id as ind1, num17 as "1_17", num18 as "1_18", num19 as "1_19", num20 as "1_20", num21 as "1_21"
FROM fact_nums
  WHERE indicator_id = 1 
  GROUP BY org_id, indicator_id),
  t2 AS (SELECT org_id, indicator_id as ind45, num17 as "45_17", num18 as "45_18", num19 as "45_19", num20 as "45_20", num21 as "45_21"
  FROM fact_nums
  WHERE indicator_id = 45
  GROUP BY org_id, indicator_id),
  t3 AS (SELECT org_id, indicator_id as ind128, num17 as "128_17", num18 as "128_18", num19 as "128_19", num20 as "128_20", num21 as "128_21"
  FROM fact_nums
  WHERE indicator_id = 128
  GROUP BY org_id, indicator_id)
  --НМА/Активы * ЧП
  SELECT DISTINCT t1.org_id,  
  ("45_17"/"1_17")*"128_17" as S_Kostya17, 
  ("45_18"/"1_18")*"128_18" as S_Kostya18, 
  ("45_19"/"1_19")*"128_19" as S_Kostya19, 
  ("45_20"/"1_20")*"128_20" as S_Kostya20,
  ("45_21"/"1_21")*"128_21" as S_Kostya21
  FROM t3 
  JOIN t2 on t3.org_id=t2.org_id
  JOIN t1 on t3.org_id=t1.org_id
  ORDER BY S_Kostya21 desc''')

,org_id,S_Kostya17,S_Kostya18,S_Kostya19,S_Kostya20,S_Kostya21
0,190,4.727883e+07,4.860566e+07,2.444295e+08,2.948443e+08,4.601509e+08
1,280,NaN,-5.462701e+07,1.761736e+07,1.840206e+08,1.654141e+08
2,8,NaN,1.624280e+08,7.338489e+07,1.399382e+08,9.944829e+07
3,141,9.095477e+07,8.018698e+07,8.903547e+07,9.374090e+07,8.339393e+07
4,49,0.000000e+00,0.000000e+00,5.055705e+04,1.897700e+07,6.305278e+07
...,...,...,...,...,...,...
160,179,4.239196e+07,1.423809e+07,NaN,NaN,NaN
161,180,4.851243e+03,-4.281021e+05,NaN,NaN,NaN
162,186,8.273117e+04,3.901146e+04,NaN,NaN,NaN
163,192,-1.684107e+05,-2.556751e+03,NaN,NaN,NaN


In [12]:
sql('''
 SELECT 
org_id, 
SUM(case when indicator_id = 45 then num17 end)/SUM(case when indicator_id = 1 then num17 end) * SUM(case when indicator_id = 128 then num17 end) as S_Kostya17,
SUM(case when indicator_id = 45 then num18 end)/SUM(case when indicator_id = 1 then num18 end) * SUM(case when indicator_id = 128 then num18 end) as S_Kostya18,
SUM(case when indicator_id = 45 then num19 end)/SUM(case when indicator_id = 1 then num19 end) * SUM(case when indicator_id = 128 then num19 end) as S_Kostya19,
SUM(case when indicator_id = 45 then num20 end)/SUM(case when indicator_id = 1 then num20 end) * SUM(case when indicator_id = 128 then num20 end) as S_Kostya20,
SUM(case when indicator_id = 45 then num21 end)/SUM(case when indicator_id = 1 then num21 end) * SUM(case when indicator_id = 128 then num21 end) as S_Kostya21
FROM fact_nums
GROUP BY org_division_id
ORDER BY S_Kostya21 desc
''')

,org_id,S_Kostya17,S_Kostya18,S_Kostya19,S_Kostya20,S_Kostya21
0,301,4.874506e+07,5.107116e+07,2.414352e+08,2.920198e+08,4.486376e+08
1,299,4.392796e+07,3.204248e+08,1.064145e+08,8.417822e+08,3.577583e+08
2,266,1.442708e+08,1.872089e+08,1.722633e+08,1.539380e+08,1.559566e+08
3,217,-3.104540e+05,2.156929e+05,-4.690502e+06,3.100931e+06,2.563691e+07
4,300,-0.000000e+00,5.521418e+03,7.031733e+06,1.096926e+07,1.884704e+07
5,302,5.274451e+06,5.087319e+07,6.669882e+06,1.623560e+07,7.799006e+06
6,303,-3.924509e+07,-2.358930e+07,3.688577e+06,-6.577090e+05,5.817010e+06
7,272,4.015936e+05,5.631386e+05,6.882998e+05,1.586043e+06,4.251141e+05
8,239,5.010229e+05,4.529481e+05,3.785102e+05,3.031835e+05,4.250031e+05
9,156,1.516295e+04,-1.166972e+05,6.700902e+05,-3.211352e+05,4.225332e+05


In [43]:
def get_indicator(indicator_id):
   df = df_fact_nums[df_fact_nums['indicator_id'] == indicator_id]
   df = df[['org_id','num17', 'num18', 'num19', 'num20', 'num21']]
   df.rename(columns = {'org_id':'org_id',
                        'num17': 'num17_'+str(indicator_id), 
                        'num18': 'num18_'+str(indicator_id),
                        'num19': 'num19_'+str(indicator_id),
                        'num20': 'num20_'+str(indicator_id),
                        'num21': 'num21_'+str(indicator_id)
                        }, inplace=True)
   df = df_dict_org.merge(df, on = 'org_id', how = 'left')
   df = df.fillna(0)
   return(df)

def get_org_indicator(indicator_id):
   df = df_fact_nums[df_fact_nums['indicator_id'] == indicator_id]
   df = df[['org_id','num17', 'num18', 'num19', 'num20', 'num21']]
   df = df_dict_org.merge(df, on = 'org_id', how = 'left')
   df = df.fillna(0)
   return(df)

def get_div_indicator(indicator_id):
   df = df_fact_nums[df_fact_nums['indicator_id'] == indicator_id]
   df = df[['org_division_id','num17', 'num18', 'num19', 'num20', 'num21']]
   df = df.groupby('org_division_id', as_index = False).sum()
   df = df.fillna(0)
   df = df.set_index('org_division_id')
   return(df)

In [27]:
res = get_org_indicator(45)/get_org_indicator(1)*get_org_indicator(128)
res.drop('org_name', axis = 1).sort_values(['num21'], ascending=False).dropna(axis=0)

,org_id,num17,num18,num19,num20,num21
184,190.0,4.727883e+07,4.860566e+07,2.444295e+08,2.948443e+08,4.601509e+08
136,141.0,9.095477e+07,8.018698e+07,8.903547e+07,9.374090e+07,8.339393e+07
48,49.0,0.000000e+00,0.000000e+00,5.055705e+04,1.897700e+07,6.305278e+07
39,40.0,2.620096e+06,6.526494e+06,3.696788e+06,1.782764e+06,5.999962e+07
133,138.0,1.750881e+05,1.220646e+06,-7.707086e+05,6.813561e+05,4.757372e+07
...,...,...,...,...,...,...
135,140.0,1.911702e+05,7.754667e+06,3.968077e+07,2.837401e+07,-1.397824e+07
156,162.0,7.674797e+04,-6.580590e+07,-2.360343e+07,-2.005195e+07,-1.480214e+07
40,41.0,-6.745865e+06,-2.440992e+07,-1.463730e+07,-1.125828e+07,-4.082781e+07
165,171.0,-1.930868e+07,-1.197491e+08,-1.109486e+08,-4.641691e+07,-9.160457e+07


In [78]:
res = get_div_indicator(128)/get_div_indicator(1)
res = res.merge(df_dict_org_division, on = 'org_division_id', how = 'left')
res = res.sort_values('num21', ascending = False)
res = res.drop('org_division_id',axis=1)
cols = res.columns.tolist()
cols = cols[-1:]+cols[:-1]
res[cols]

,org_division_name,num17,num18,num19,num20,num21
17,Сбыт и трейдинг,0.125917,0.189439,0.168601,0.177820,0.267130
21,Электроэнергетический,0.026996,0.022103,0.049654,0.046510,0.064396
19,Топливный,0.071284,0.076221,0.073534,0.065910,0.055671
3,Аварийная готовность и специальные перевозки,0.000190,0.093849,0.041352,0.026367,0.035984
9,Инжиниринговый,-0.009829,0.002957,-0.053973,0.026699,0.022494
16,РИР,-0.000289,0.002315,0.016349,0.016878,0.018701
5,АХД и сервисная инфраструктура,0.016634,0.029961,0.006526,0.045158,0.017769
2,Rusatom Healthcare,-0.082584,-0.034817,0.006172,-0.001524,0.017219
10,Машиностроительный,0.009328,0.046725,0.006646,0.027307,0.012194
22,Ядерный оружейный комплекс,0.012271,0.013011,0.007134,0.004706,0.009583


In [69]:
sql('''

SELECT 
fn.org_division_id, 
dod.org_division_name,
SUM(case when fn.indicator_id = 128 then fn.num17 end)/SUM(case when fn.indicator_id = 1 then fn.num17 end) as NPtoA_17,
SUM(case when fn.indicator_id = 128 then fn.num18 end)/SUM(case when fn.indicator_id = 1 then fn.num18 end) as NPtoA_18,
SUM(case when fn.indicator_id = 128 then fn.num19 end)/SUM(case when fn.indicator_id = 1 then fn.num19 end) as NPtoA_19,
SUM(case when fn.indicator_id = 128 then fn.num20 end)/SUM(case when fn.indicator_id = 1 then fn.num20 end) as NPtoA_20,
SUM(case when fn.indicator_id = 128 then fn.num21 end)/SUM(case when fn.indicator_id = 1 then fn.num21 end) as NPtoA_21

FROM fact_nums fn
JOIN dict_org_division dod ON fn.org_division_id = dod.org_division_id
GROUP BY fn.org_division_id, dod.org_division_name
ORDER BY NPtoA_21 desc
''')

,org_division_id,org_division_name,NPtoA_17,NPtoA_18,NPtoA_19,NPtoA_20,NPtoA_21
0,18,Сбыт и трейдинг,0.125917,0.189439,0.168601,0.177820,0.267130
1,22,Электроэнергетический,0.026996,0.022103,0.049654,0.046510,0.064396
2,20,Топливный,0.071284,0.076221,0.073534,0.065910,0.055671
3,3,Аварийная готовность и специальные перевозки,0.000190,0.093849,0.041352,0.026367,0.035984
4,9,Инжиниринговый,-0.009829,0.002957,-0.053973,0.026699,0.022494
5,17,РИР,-0.000289,0.002315,0.016349,0.016878,0.018701
6,5,АХД и сервисная инфраструктура,0.016634,0.029961,0.006526,0.045158,0.017769
7,2,Rusatom Healthcare,-0.082584,-0.034817,0.006172,-0.001524,0.017219
8,11,Машиностроительный,0.009328,0.046725,0.006646,0.027307,0.012194
9,23,Ядерный оружейный комплекс,0.012271,0.013011,0.007134,0.004706,0.009583


In [16]:
df = df_fact_nums.merge(df_dict_indicator).merge(df_dict_indicator_type).merge(df_dict_statement).iloc[:,4:]
print(df.columns)
df[['indicator_name','indicator_type_name', 'statement_name', 'num17', 'num18', 'num19', 'num20', 'num21']].head()

Index(['statement_id', 'num17', 'num18', 'num19', 'num20', 'num21',
       'indicator_name', 'indicator_type_name', 'statement_name'],
      dtype='object')


,indicator_name,indicator_type_name,statement_name,num17,num18,num19,num20,num21
0,Cooтношение дебиторской задолженности к актива...,%,Нет,0.0564,0.0836,0.1115,0.0374,0.1636
1,Cooтношение дебиторской задолженности к актива...,%,Нет,0.0000,0.0000,0.0000,0.0000,0.7701
2,Cooтношение дебиторской задолженности к актива...,%,Нет,0.0000,0.0000,0.0000,0.0000,0.0119
3,Cooтношение дебиторской задолженности к актива...,%,Нет,0.0000,0.0000,0.0000,0.3443,0.2802
4,Cooтношение дебиторской задолженности к актива...,%,Нет,0.0000,0.0000,0.2693,0.0211,0.0653


In [17]:
sql('''
select * from fact_nums
limit 10
''')

,org_id,org_division_id,indicator_id,indicator_type_id,statement_id,num17,num18,num19,num20,num21
0,1,22,129,1,0,5.640000e-02,8.360000e-02,1.115000e-01,3.740000e-02,1.636000e-01
1,1,22,1,0,1,1.267406e+09,1.337535e+09,1.495419e+09,1.524282e+09,1.964642e+09
2,1,22,2,0,2,5.039000e+06,3.279700e+07,5.956000e+06,6.881000e+06,1.106100e+07
3,1,22,3,1,0,2.950000e-02,1.725000e-01,2.890000e-02,5.600000e-02,2.750000e-02
4,1,22,4,1,0,1.607000e-01,1.097000e+00,2.123000e-01,1.888000e-01,-3.579000e-01
5,1,22,5,1,0,2.860000e-02,1.471000e-01,2.810000e-02,5.300000e-02,2.680000e-02
6,1,22,6,0,1,9.115240e+08,9.086390e+08,1.008689e+09,1.057311e+09,1.077820e+09
7,1,22,10,0,4,6.650100e+07,4.747520e+08,7.000000e+03,0.000000e+00,0.000000e+00
8,1,22,11,0,2,1.760570e+08,2.229080e+08,2.117240e+08,1.297490e+08,4.130740e+08
9,1,22,12,0,1,7.141900e+07,1.118560e+08,1.667260e+08,5.702300e+07,3.213350e+08


In [18]:
df_fact_nums.head(10)

,org_id,org_division_id,indicator_id,indicator_type_id,statement_id,num17,num18,num19,num20,num21
0,1,22,129,1,0,5.640000e-02,8.360000e-02,1.115000e-01,3.740000e-02,1.636000e-01
1,1,22,1,0,1,1.267406e+09,1.337535e+09,1.495419e+09,1.524282e+09,1.964642e+09
2,1,22,2,0,2,5.039000e+06,3.279700e+07,5.956000e+06,6.881000e+06,1.106100e+07
3,1,22,3,1,0,2.950000e-02,1.725000e-01,2.890000e-02,5.600000e-02,2.750000e-02
4,1,22,4,1,0,1.607000e-01,1.097000e+00,2.123000e-01,1.888000e-01,-3.579000e-01
5,1,22,5,1,0,2.860000e-02,1.471000e-01,2.810000e-02,5.300000e-02,2.680000e-02
6,1,22,6,0,1,9.115240e+08,9.086390e+08,1.008689e+09,1.057311e+09,1.077820e+09
7,1,22,10,0,4,6.650100e+07,4.747520e+08,7.000000e+03,0.000000e+00,0.000000e+00
8,1,22,11,0,2,1.760570e+08,2.229080e+08,2.117240e+08,1.297490e+08,4.130740e+08
9,1,22,12,0,1,7.141900e+07,1.118560e+08,1.667260e+08,5.702300e+07,3.213350e+08
